In [ ]:
# Questäo 1)a)
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

import pandas as pd
ds_inicial = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Agrupar os dados por animal e lactação
grouped = ds_inicial.groupby(['NUMERO', 'lactation'])

# Calcular os valores mínimos, médios e máximos para cada grupo
result = grouped['PRODUCAO_PESO'].agg(['min', 'mean', 'max'])

result = result.round(2)
result = result.sort_values(by=['NUMERO'])

# Fazer tabela
tabela_vis = result.style.format("{:.2f}".format).set_properties(**{'background-color': 'white',
                                            'color': 'black',
                                            'border-color': 'black'}) \
                        .set_table_styles([{'selector': 'th',
                                           'props': [('font-size', '12px'),
                                                     ('text-align', 'center')]}])

# Exibindo a tabela
tabela_vis

In [ ]:
#Questão 1)b)
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount= True)

ds_inicial = pd.read_csv("/content/drive/MyDrive/dataset.csv")

def plotar_regressao(grupo, titulo):
    x = grupo['dim']
    y = grupo['PRODUCAO_PESO']
    a, b, eqm = ajuste_linear(x, y)

    # Verificar se a regressão é válida
    if a is None or b is None:
        print(f"Não é possivel fazer o gráfico de: {titulo} devido ao resultado invalido.")
        return
    # Fazendo o gráfico
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, label='Dados')
    plt.plot(x, a*x + b, color='red', label='Reta de Regressão')
    plt.xlabel('Dias em lactação')
    plt.ylabel('Produção por peso')
    plt.title(titulo)
    plt.legend()
    plt.grid(True)
    plt.show()

# Função para realizar o ajuste linear
def ajuste_linear(x, y):
    n = len(x)
    soma_x = sum(x)
    soma_y = sum(y)
    soma_x2 = sum(x**2)
    soma_xy = sum(x * y)

# Verificar se a variância de x é diferente de zero(não pode divisão por zero)
    if np.var(x) == 0:
        print("Todos os valores de x são iguais. A regressão linear não é apropriada.")
        return None, None, None

    a = (n * soma_xy - soma_x * soma_y) / (n * soma_x2 - soma_x**2)
    b = (soma_y - a * soma_x) / n

    y_pred = a * x + b
    error = y - y_pred
    mse = np.mean(error**2)
    return a, b, mse

# Agrupar os dados por animal e lactação
grouped = ds_inicial.groupby(['NUMERO', 'lactation'])

# Criar uma lista para armazenar os resultados
resultados = []

# Iterar sobre os grupos e realizar o ajuste
for (NUMERO, lactation), grupo in grouped:
    x = grupo['dim']
    y = grupo['PRODUCAO_PESO']
    a, b, mse = ajuste_linear(x, y)
    resultados.append({'NUMERO': NUMERO, 'lactation': lactation, 'a': a, 'b': b, 'EQM': mse})


# Converter a lista para um DataFrame
resultado_erro = pd.DataFrame(resultados)

# Organizando o Data Frame
resultado_erro = resultado_erro.sort_values(by=['NUMERO'])
resultado_erro = resultado_erro.dropna()

# Exibir o DataFrame
pd.set_option('display.max_rows', None)
print(resultado_erro)


for (NUMERO, lactation), grupo in grouped:
    titulo = f'Animal {NUMERO}, Lactação {lactation}'
    plotar_regressao(grupo, titulo)

In [ ]:
#Questão 1)c)
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount= True)

ds_inicial = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Função para realizar o ajuste linear
def ajuste_linear(x, y):
    n = len(x)
    soma_x = sum(x)
    soma_y = sum(y)
    soma_x2 = sum(x**2)
    soma_xy = sum(x * y)

# Verificar se a variância de x é diferente de zero(não pode divisão por zero)
    if np.var(x) == 0:
        print("Todos os valores de x são iguais. A regressão linear não é apropriada.")
        return None, None, None

    a = (n * soma_xy - soma_x * soma_y) / (n * soma_x2 - soma_x**2)
    b = (soma_y - a * soma_x) / n

    y_pred = a * x + b
    error = y - y_pred
    mse = np.mean(error**2)
    return a, b, mse

# Lista para armazenar os resultados
resultados = []

# Iterar sobre cada vaca e lactação
for vaca in ds_inicial['NUMERO'].unique():
    for lactacao in ds_inicial['lactation'].unique():
        # Filtrar os dados do animal e da lactação
        dados_vaca = ds_inicial[(ds_inicial['NUMERO'] == vaca) & (ds_inicial['lactation'] == lactacao)]
        x = dados_vaca ['dim']
        y = dados_vaca ['PRODUCAO_PESO']

        # Verificar se há dados sufientes para regressao linear
        if len(x) <= 1:
            print(f"Nao há dados sufiente da {vaca} na lactacao {lactacao}")
            continue

        # Ajustar o modelo
        a, b, mse = ajuste_linear(x, y)

        # Prever a produção para outros animais
        for vaca_alvo in ds_inicial['NUMERO'].unique():
            if vaca_alvo != vaca:
                dados_vaca_alvo = ds_inicial[(ds_inicial['NUMERO'] == vaca_alvo) & (ds_inicial['lactation'] == lactacao)]
                x_alvo = dados_vaca_alvo['dim']
                y_alvo = dados_vaca_alvo['PRODUCAO_PESO']

                y_pred = a*x_alvo + b
                mse_teste = np.mean((y_alvo - y_pred)**2)

                resultados.append([vaca, lactacao, vaca_alvo, y_pred, mse_teste])

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados, columns=['Vaca Utilizada', 'Lactação', 'Vaca Alvo', 'Produção Alvo', 'Erro Quadrático Médio'])

# Exibir o DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df_resultados)


In [ ]:
#Questão 1)d)
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount= True)

ds_inicial = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# Função para realizar o ajuste linear
def ajuste_linear(x, y):
    n = len(x)
    soma_x = sum(x)
    soma_y = sum(y)
    soma_x2 = sum(x**2)
    soma_xy = sum(x * y)

# Verificar se a variância de x é diferente de zero(não pode divisão por zero)
    if np.var(x) == 0:
        print("Todos os valores de x são iguais. A regressão linear não é apropriada.")
        return None, None, None

    a = (n * soma_xy - soma_x * soma_y) / (n * soma_x2 - soma_x**2)
    b = (soma_y - a * soma_x) / n

    y_pred = a * x + b
    error = y - y_pred
    mse = np.mean(error**2)
    return a, b, mse

# Lista para armazenar os resultados
resultados = []

# Iterar sobre cada vaca e lactação
for vaca in ds_inicial['NUMERO'].unique():
    for lactacao in ds_inicial['lactation'].unique():
        # Filtrar os dados do animal e da lactação
        dados_vaca = ds_inicial[(ds_inicial['NUMERO'] == vaca) & (ds_inicial['lactation'] == lactacao)]
        x = dados_vaca ['dim']
        y = dados_vaca ['PRODUCAO_PESO']

        # Verificar se há dados sufientes para regressao linear
        if len(x) <= 1:
            print(f"Nao há dados sufiente da {vaca} na lactacao {lactacao}")
            continue

        # Ajustar o modelo
        a, b, mse = ajuste_linear(x, y)

         # Prever a produção para outros animais
        for vaca_alvo in ds_inicial['NUMERO'].unique():
            if vaca_alvo != vaca:
                dados_vaca_alvo = ds_inicial[(ds_inicial['NUMERO'] == vaca_alvo) & (ds_inicial['lactation'] == lactacao)]
                x_alvo = dados_vaca_alvo['dim']
                y_alvo = dados_vaca_alvo['PRODUCAO_PESO']

                y_pred = a*x_alvo + b
                mse_teste = np.mean((y_alvo - y_pred)**2)
                resultados.append([vaca, lactacao, vaca_alvo, y_pred, mse_teste])


df_resultados = pd.DataFrame(resultados, columns=['Vaca Utilizada', 'Lactação', 'Vaca Alvo', 'Produção Alvo', 'Erro Quadrático Médio'])

tabela_mse = pd.pivot_table(
    df_resultados,
    values='Erro Quadrático Médio',
    index='Vaca Utilizada',
    columns=['Lactação', 'Vaca Alvo'],
    aggfunc='mean'
)
# Agrupar por lactação e encontrar o menor valor de MSE para cada grupo
menor_mse_por_lactacao = tabela_mse.groupby(level='Lactação', axis=1).min()

# Criar um DataFrame para os resultados
resultado = pd.DataFrame({'Animal com Menor MSE': menor_mse_por_lactacao.idxmin(axis=0)})

# Exibir os resultados
print(resultado)

Mounted at /content/drive
Nao há dados sufiente da 1503 na lactacao 5
Nao há dados sufiente da 1517 na lactacao 5
Nao há dados sufiente da 1527 na lactacao 4
Nao há dados sufiente da 1527 na lactacao 5
Nao há dados sufiente da 1555 na lactacao 4
Nao há dados sufiente da 1555 na lactacao 5
Nao há dados sufiente da 1564 na lactacao 4
Nao há dados sufiente da 1564 na lactacao 5
Nao há dados sufiente da 1566 na lactacao 5
Nao há dados sufiente da 1573 na lactacao 4
Nao há dados sufiente da 1573 na lactacao 5
Nao há dados sufiente da 1584 na lactacao 5
Nao há dados sufiente da 1593 na lactacao 5
Nao há dados sufiente da 1594 na lactacao 5
Nao há dados sufiente da 1622 na lactacao 5
Nao há dados sufiente da 1624 na lactacao 4
Nao há dados sufiente da 1624 na lactacao 5
Nao há dados sufiente da 1635 na lactacao 4
Nao há dados sufiente da 1635 na lactacao 5
Nao há dados sufiente da 1638 na lactacao 4
Nao há dados sufiente da 1638 na lactacao 5
Nao há dados sufiente da 1645 na lactacao 4
Nao há

<ipython-input-13-6a4493f59a1e>:72: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  menor_mse_por_lactacao = tabela_mse.groupby(level='Lactação', axis=1).min()
